<a href="https://colab.research.google.com/github/chiahsuy/2014/blob/master/Copy_of_FRET_TextProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Imports

In [ ]:
import pandas as pd
import numpy as np

!pip install dfply
from dfply import *

     |████████████████████████████████| 612 kB 7.9 MB/s 


In [ ]:
import datetime
from tqdm import tqdm
import requests

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# pd.options.display.max_colwidth = 600

import random
random.seed(9)

#### Get Data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Set gdrive path
gdrive_path = "/content/gdrive/MyDrive/FRET/"

#### Upload Text

In [ ]:
pd.set_option('display.max_colwidth', None)

test_cases = pd.read_csv(gdrive_path + "FRET_Test.csv") 
test_cases.columns = map(str.lower, test_cases.columns)
print(test_cases.shape)
test_cases.head()

(50, 2)


,id,text
0,21,Customer stated that he purchased a bike with a warranty. The breaks on his bike broke. Ths is highly dangerous.
1,22,Customer purchased a sealed pack of strawberries. Found insects inside after opening the pack. Disgusting and unsanitary.
2,23,I purchased a microwave. It started smoking after a week. We were scared it may catch fire.
3,24,"Customer states the measuring cups in ABC cookware were razor sharp and the customer has cut herself on them several times, and is now afraid to use them."
4,25,"I bought a mat for the bottom of my tub, but fell twice in the tub and banged my knee because it slips. It is supposed to stick. I have been severely injured."


#### Text Cleaning Steps



1.   Remove punctuation
2.   Remove numbers
3.   Make everything lowercase
4.   Remove stopwords
5.   Lemmatize
6.   Keep only unique words in a string (optional)





In [ ]:
import nltk
# nltk.download('all')
import string # for list of all punctuations
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re # importing Regex

In [ ]:
test_cases >>= mutate(clean_text = X.text.str.replace('[^\w\s]',' ')) # Remove Punctuations
test_cases >>= mutate(clean_text = X.clean_text.str.replace('\d+','')) # Remove Numbers
test_cases >>= mutate(clean_text = X.clean_text.str.lower()) # Lowercase

test_cases.head()

,id,text,clean_text
0,21,Customer stated that he purchased a bike with a warranty. The breaks on his bike broke. Ths is highly dangerous.,customer stated that he purchased a bike with a warranty the breaks on his bike broke ths is highly dangerous
1,22,Customer purchased a sealed pack of strawberries. Found insects inside after opening the pack. Disgusting and unsanitary.,customer purchased a sealed pack of strawberries found insects inside after opening the pack disgusting and unsanitary
2,23,I purchased a microwave. It started smoking after a week. We were scared it may catch fire.,i purchased a microwave it started smoking after a week we were scared it may catch fire
3,24,"Customer states the measuring cups in ABC cookware were razor sharp and the customer has cut herself on them several times, and is now afraid to use them.",customer states the measuring cups in abc cookware were razor sharp and the customer has cut herself on them several times and is now afraid to use them
4,25,"I bought a mat for the bottom of my tub, but fell twice in the tub and banged my knee because it slips. It is supposed to stick. I have been severely injured.",i bought a mat for the bottom of my tub but fell twice in the tub and banged my knee because it slips it is supposed to stick i have been severely injured


In [ ]:
#@title Remove Stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))

# stopwords_set.pop()
# for i, val in enumerate(random.sample(stopwords_set, 10)): 
#   print(val)

import time
tic = time.perf_counter()
test_cases['clean_text_wout_stopwords'] = pd.Series([[w for w in text.split() if w.lower() not in stopwords_set]
            for text in test_cases['clean_text']]).apply(lambda k : ' '.join(k))
toc = time.perf_counter()
print(f"Completed in : {toc - tic:0.4f} seconds")

test_cases.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Completed in : 0.0030 seconds


,id,text,clean_text,clean_text_wout_stopwords
0,21,Customer stated that he purchased a bike with a warranty. The breaks on his bike broke. Ths is highly dangerous.,customer stated that he purchased a bike with a warranty the breaks on his bike broke ths is highly dangerous,customer stated purchased bike warranty breaks bike broke ths highly dangerous
1,22,Customer purchased a sealed pack of strawberries. Found insects inside after opening the pack. Disgusting and unsanitary.,customer purchased a sealed pack of strawberries found insects inside after opening the pack disgusting and unsanitary,customer purchased sealed pack strawberries found insects inside opening pack disgusting unsanitary
2,23,I purchased a microwave. It started smoking after a week. We were scared it may catch fire.,i purchased a microwave it started smoking after a week we were scared it may catch fire,purchased microwave started smoking week scared may catch fire
3,24,"Customer states the measuring cups in ABC cookware were razor sharp and the customer has cut herself on them several times, and is now afraid to use them.",customer states the measuring cups in abc cookware were razor sharp and the customer has cut herself on them several times and is now afraid to use them,customer states measuring cups abc cookware razor sharp customer cut several times afraid use
4,25,"I bought a mat for the bottom of my tub, but fell twice in the tub and banged my knee because it slips. It is supposed to stick. I have been severely injured.",i bought a mat for the bottom of my tub but fell twice in the tub and banged my knee because it slips it is supposed to stick i have been severely injured,bought mat bottom tub fell twice tub banged knee slips supposed stick severely injured


In [ ]:
#@title Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.tokenize import sent_tokenize, word_tokenize

# print("tested : ", wordnet_lemmatizer.lemmatize("tested", pos = "v"))
# print("tests : ", wordnet_lemmatizer.lemmatize("tested", pos = "v"), "\n")
# print("drowned : ", wordnet_lemmatizer.lemmatize("drowned", pos = "v"))
# print("drowning : ", wordnet_lemmatizer.lemmatize("drowning", pos = "v"))


import time
tic = time.perf_counter()
test_cases['clean_text_lem'] = pd.Series([[wordnet_lemmatizer.lemmatize(w, pos = "v") for w in text.split()] 
                                            for text in test_cases['clean_text_wout_stopwords']]).apply(lambda k : ' '.join(k))
toc = time.perf_counter()
print(f"Completed in : {toc - tic:0.4f} seconds")

test_cases.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Completed in : 0.0038 seconds


,id,text,clean_text,clean_text_wout_stopwords,clean_text_lem
0,21,Customer stated that he purchased a bike with a warranty. The breaks on his bike broke. Ths is highly dangerous.,customer stated that he purchased a bike with a warranty the breaks on his bike broke ths is highly dangerous,customer stated purchased bike warranty breaks bike broke ths highly dangerous,customer state purchase bike warranty break bike break ths highly dangerous
1,22,Customer purchased a sealed pack of strawberries. Found insects inside after opening the pack. Disgusting and unsanitary.,customer purchased a sealed pack of strawberries found insects inside after opening the pack disgusting and unsanitary,customer purchased sealed pack strawberries found insects inside opening pack disgusting unsanitary,customer purchase seal pack strawberries find insects inside open pack disgust unsanitary
2,23,I purchased a microwave. It started smoking after a week. We were scared it may catch fire.,i purchased a microwave it started smoking after a week we were scared it may catch fire,purchased microwave started smoking week scared may catch fire,purchase microwave start smoke week scar may catch fire
3,24,"Customer states the measuring cups in ABC cookware were razor sharp and the customer has cut herself on them several times, and is now afraid to use them.",customer states the measuring cups in abc cookware were razor sharp and the customer has cut herself on them several times and is now afraid to use them,customer states measuring cups abc cookware razor sharp customer cut several times afraid use,customer state measure cup abc cookware razor sharp customer cut several time afraid use
4,25,"I bought a mat for the bottom of my tub, but fell twice in the tub and banged my knee because it slips. It is supposed to stick. I have been severely injured.",i bought a mat for the bottom of my tub but fell twice in the tub and banged my knee because it slips it is supposed to stick i have been severely injured,bought mat bottom tub fell twice tub banged knee slips supposed stick severely injured,buy mat bottom tub fell twice tub bang knee slip suppose stick severely injure


In [ ]:
#@title Keep only unique words in a string

import time
tic = time.perf_counter()
test_cases['clean_text_lem_unique'] = pd.Series([set(x.split(' ')) for x in test_cases['clean_text_lem']]).apply(lambda k : ' '.join(k))
toc = time.perf_counter()
print(f"Completed in : {toc - tic:0.4f} seconds")

test_cases.head()

Completed in : 0.0037 seconds


,id,text,clean_text,clean_text_wout_stopwords,clean_text_lem,clean_text_lem_unique
0,21,Customer stated that he purchased a bike with a warranty. The breaks on his bike broke. Ths is highly dangerous.,customer stated that he purchased a bike with a warranty the breaks on his bike broke ths is highly dangerous,customer stated purchased bike warranty breaks bike broke ths highly dangerous,customer state purchase bike warranty break bike break ths highly dangerous,dangerous purchase customer break ths warranty highly state bike
1,22,Customer purchased a sealed pack of strawberries. Found insects inside after opening the pack. Disgusting and unsanitary.,customer purchased a sealed pack of strawberries found insects inside after opening the pack disgusting and unsanitary,customer purchased sealed pack strawberries found insects inside opening pack disgusting unsanitary,customer purchase seal pack strawberries find insects inside open pack disgust unsanitary,insects purchase customer strawberries open disgust unsanitary pack seal find inside
2,23,I purchased a microwave. It started smoking after a week. We were scared it may catch fire.,i purchased a microwave it started smoking after a week we were scared it may catch fire,purchased microwave started smoking week scared may catch fire,purchase microwave start smoke week scar may catch fire,start week scar catch purchase may fire microwave smoke
3,24,"Customer states the measuring cups in ABC cookware were razor sharp and the customer has cut herself on them several times, and is now afraid to use them.",customer states the measuring cups in abc cookware were razor sharp and the customer has cut herself on them several times and is now afraid to use them,customer states measuring cups abc cookware razor sharp customer cut several times afraid use,customer state measure cup abc cookware razor sharp customer cut several time afraid use,cookware sharp cup razor customer abc cut measure afraid time state use several
4,25,"I bought a mat for the bottom of my tub, but fell twice in the tub and banged my knee because it slips. It is supposed to stick. I have been severely injured.",i bought a mat for the bottom of my tub but fell twice in the tub and banged my knee because it slips it is supposed to stick i have been severely injured,bought mat bottom tub fell twice tub banged knee slips supposed stick severely injured,buy mat bottom tub fell twice tub bang knee slip suppose stick severely injure,bottom mat bang injure knee twice slip stick buy fell tub severely suppose


We are not going to use clean_text_lem_unique since it disrupts the flow of a sentence which is required for sentiment analysis

#### Sentiment Analysis

In [ ]:
## Define a function which can be applied to calculate the score for the whole dataset
from textblob import TextBlob

def senti(x):
    return TextBlob(x).sentiment  

def polarity(x):
    return TextBlob(x).sentiment.polarity

def subjectivity(x):
    return TextBlob(x).sentiment.subjectivity


In [ ]:
# test_cases["sentiment_polarity"] = test_cases['clean_text_lem'].apply(polarity) # Lies b/w -1 to +1
# test_cases["sentiment_objectivity"] = test_cases['clean_text_lem'].apply(subjectivity) # Lies b/w 0 to 1

test_cases["sentiment_polarity"] = test_cases['clean_text_wout_stopwords'].apply(polarity) # Lies b/w -1 to +1
test_cases["sentiment_objectivity"] = test_cases['clean_text_wout_stopwords'].apply(subjectivity) # Lies b/w 0 to 1


test_cases.head(6)

,id,text,clean_text,clean_text_wout_stopwords,clean_text_lem,sentiment_polarity,sentiment_objectivity
0,21,Customer stated that he purchased a bike with a warranty. The breaks on his bike broke. Ths is highly dangerous.,customer stated that he purchased a bike with a warranty the breaks on his bike broke ths is highly dangerous,customer stated purchased bike warranty breaks bike broke ths highly dangerous,customer state purchase bike warranty break bike break ths highly dangerous,-0.60,0.90
1,22,Customer purchased a sealed pack of strawberries. Found insects inside after opening the pack. Disgusting and unsanitary.,customer purchased a sealed pack of strawberries found insects inside after opening the pack disgusting and unsanitary,customer purchased sealed pack strawberries found insects inside opening pack disgusting unsanitary,customer purchase seal pack strawberries find insects inside open pack disgust unsanitary,-1.00,1.00
2,23,I purchased a microwave. It started smoking after a week. We were scared it may catch fire.,i purchased a microwave it started smoking after a week we were scared it may catch fire,purchased microwave started smoking week scared may catch fire,purchase microwave start smoke week scar may catch fire,0.00,0.00
3,24,"Customer states the measuring cups in ABC cookware were razor sharp and the customer has cut herself on them several times, and is now afraid to use them.",customer states the measuring cups in abc cookware were razor sharp and the customer has cut herself on them several times and is now afraid to use them,customer states measuring cups abc cookware razor sharp customer cut several times afraid use,customer state measure cup abc cookware razor sharp customer cut several time afraid use,-0.24,0.55
4,25,"I bought a mat for the bottom of my tub, but fell twice in the tub and banged my knee because it slips. It is supposed to stick. I have been severely injured.",i bought a mat for the bottom of my tub but fell twice in the tub and banged my knee because it slips it is supposed to stick i have been severely injured,bought mat bottom tub fell twice tub banged knee slips supposed stick severely injured,buy mat bottom tub fell twice tub bang knee slip suppose stick severely injure,0.00,0.00
5,26,The cord of this phone charger is coated with metal and as it was charging the metal part almost caught fire. It ignited in the socket and burned out the outlet in the wall. I caught it in time. Product is unsafe to use.,the cord of this phone charger is coated with metal and as it was charging the metal part almost caught fire it ignited in the socket and burned out the outlet in the wall i caught it in time product is unsafe to use,cord phone charger coated metal charging metal part almost caught fire ignited socket burned outlet wall caught time product unsafe use,cord phone charger coat metal charge metal part almost catch fire ignite socket burn outlet wall catch time product unsafe use,0.00,0.00


#### Numeric Representation of Text

In [ ]:
import csv
glove_vectors = pd.read_table(gdrive_path + "glove.6B.50d.txt", sep = " ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
glove_vectors.index.names = ['words']
glove_vectors.shape

(400000, 50)

In [ ]:
glove_vectors.head(3)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
words,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
the,0.42,0.25,-0.41,0.12,0.35,-0.04,-0.50,-0.18,-0.00,-0.66,0.28,-0.15,-0.56,0.15,-0.01,0.01,0.10,-0.13,-0.84,-0.12,-0.02,-0.33,-0.16,-0.23,-0.19,-1.88,-0.77,0.10,-0.42,-0.20,4.01,-0.19,-0.52,-0.32,0.00,0.01,0.18,-0.16,0.01,-0.05,-0.30,-0.16,-0.35,-0.05,-0.44,0.19,0.00,-0.18,-0.12,-0.79
",",0.01,0.24,-0.17,0.41,0.64,0.48,-0.43,-0.56,-0.36,-0.24,0.13,-0.06,-0.40,-0.48,0.23,0.09,-0.13,0.08,-0.42,-0.15,0.10,0.49,0.31,-0.13,-0.04,-1.52,0.13,-0.02,-0.04,-0.28,3.54,-0.12,-0.01,-0.15,0.22,-0.33,-0.14,0.32,0.70,0.45,-0.08,0.63,0.32,-0.47,0.23,0.36,-0.38,-0.57,0.04,0.30
.,0.15,0.30,-0.17,0.18,0.32,0.34,-0.43,-0.31,-0.45,-0.29,0.17,0.12,-0.41,-0.42,0.60,0.29,-0.12,-0.04,-0.68,-0.25,0.18,0.09,0.47,0.02,0.04,-1.47,-0.30,-0.02,0.31,-0.22,3.75,0.00,-0.18,-0.46,0.10,-0.12,0.24,0.12,0.42,0.06,-0.00,0.07,0.09,-0.10,-0.14,0.22,-0.08,-0.36,0.02,0.10


In [ ]:
gloveWord_set = set(glove_vectors.index)

## for assets
import time
tic = time.perf_counter()
test_cases['clean_text_glvwrds'] = pd.Series([[w for w in text.split() if w.lower() in gloveWord_set]
            for text in test_cases['clean_text_lem']]).apply(lambda k : ' '.join(k))
toc = time.perf_counter()
print(f"Completed in : {toc - tic:0.4f} seconds")

Completed in : 0.0325 seconds


In [ ]:
print("How many reviews did not match any work with the loaded glove corpus?", sum(test_cases['clean_text_glvwrds'].isna()), "\n")
test_cases.head(1)

How many reviews did not match any work with the loaded glove corpus? 0 



,id,text,clean_text,clean_text_wout_stopwords,clean_text_lem,sentiment_polarity,sentiment_objectivity,clean_text_glvwrds
0,21,Customer stated that he purchased a bike with a warranty. The breaks on his bike broke. Ths is highly dangerous.,customer stated that he purchased a bike with a warranty the breaks on his bike broke ths is highly dangerous,customer stated purchased bike warranty breaks bike broke ths highly dangerous,customer state purchase bike warranty break bike break ths highly dangerous,-0.60,0.90,customer state purchase bike warranty break bike break ths highly dangerous


In [ ]:
from itertools import chain

tic = time.perf_counter()

temp_id = list(chain(*[[c]*len(x.split()) for c, x in enumerate(test_cases['clean_text_glvwrds'], 1)]))
temp_words = list(chain(*[x.split() for x in test_cases['clean_text_glvwrds']]))
emb_temp = glove_vectors.loc[temp_words]
emb_temp['id'] = temp_id
del temp_id, temp_words
embedding_text = emb_temp.groupby(["id"], as_index = False)[emb_temp.columns[:-1]].mean()
embedding_text.rename(columns = {'id' : 'clean_text_glvwrds'}, inplace = True)
embedding_text['clean_text_glvwrds'] = test_cases['clean_text_glvwrds']

toc = time.perf_counter()
print(f"Completed the Task in {toc - tic:0.4f} seconds")

Completed the Task in 0.1121 seconds


In [ ]:
embedding_text.head()

,clean_text_glvwrds,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,customer state purchase bike warranty break bike break ths highly dangerous,0.07,0.10,0.14,0.13,-0.13,0.16,-0.37,0.04,0.42,0.07,-0.03,-0.14,-0.27,-0.05,0.33,0.04,-0.08,0.31,0.02,-0.61,0.14,-0.15,-0.43,0.24,0.05,-1.05,0.01,0.09,0.74,-0.08,2.16,0.14,-0.15,0.11,0.23,0.01,0.09,0.25,-0.19,-0.16,-0.09,-0.24,0.16,0.48,-0.10,-0.30,0.18,0.11,0.02,0.17
1,customer purchase seal pack strawberries find insects inside open pack disgust unsanitary,0.49,-0.08,0.08,-0.09,0.36,0.06,-0.50,-0.02,0.31,-0.02,0.02,0.03,0.31,0.17,0.39,0.25,-0.01,0.00,-0.18,-0.71,0.15,-0.07,0.31,0.17,-0.10,-0.68,-0.12,0.38,0.70,-0.23,1.94,0.34,-0.18,-0.06,-0.08,0.39,-0.15,0.15,0.17,-0.41,0.20,-0.14,0.07,0.48,0.44,0.06,-0.00,-0.22,0.16,-0.22
2,purchase microwave start smoke week scar may catch fire,0.40,0.01,0.37,-0.19,0.01,0.15,-0.49,0.05,0.16,-0.03,-0.06,-0.13,-0.05,0.25,0.31,0.53,-0.16,-0.18,-0.82,-0.50,0.37,0.06,0.31,-0.18,0.00,-1.14,-0.01,0.33,0.82,-0.18,2.41,0.23,-0.40,-0.12,0.01,0.12,0.25,-0.02,0.37,-0.05,0.02,0.13,-0.09,0.25,0.31,0.28,0.22,0.04,-0.17,-0.08
3,customer state measure cup abc cookware razor sharp customer cut several time afraid use,-0.03,-0.06,0.31,0.16,0.08,0.33,-0.28,0.01,-0.03,-0.16,-0.10,-0.09,-0.12,-0.02,0.22,0.23,-0.11,0.00,-0.05,-0.76,0.40,-0.23,0.13,0.04,-0.13,-1.22,-0.29,0.14,0.38,-0.19,2.53,0.33,0.03,-0.11,-0.08,-0.07,-0.08,0.33,-0.02,-0.37,0.10,0.15,0.01,0.13,0.06,0.02,-0.09,0.19,-0.04,0.30
4,buy mat bottom tub fell twice tub bang knee slip suppose stick severely injure,-0.08,-0.16,0.34,-0.23,0.09,0.21,0.09,0.14,-0.13,0.01,0.08,0.12,-0.44,0.27,0.20,0.56,0.05,-0.12,-0.33,-0.48,-0.04,0.10,0.28,-0.13,0.13,-0.78,-0.04,0.54,0.58,-0.35,1.78,0.47,0.16,0.61,-0.03,0.25,0.43,0.20,0.56,-0.29,-0.01,-0.01,0.05,0.31,0.21,0.06,0.11,-0.47,0.35,-0.21
